# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [29]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [30]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [31]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [32]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [33]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [34]:
try:
    session.execute(""" 
    CREATE KEYSPACE IF NOT EXISTS sparkfly
    WITH REPLICATION= 
    {'class': 'SimpleStrategy', 'replication_factor':1}"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [35]:
try:
    session.set_keyspace('sparkfly')
except Exception as e:
    print(e)

### import data from file into a pandas dataframe

In [38]:
file = 'event_datafile_new.csv'
df = pd.read_csv('event_datafile_new.csv')
df['user']= df['firstName'] + ' ' + df['lastName'] 

# Task1: Getting artist, song name and lenght through session ID and Item in session
our prmary key is a composite key (SessionId and itemInSession) because of the desired result, as with nosql databases like Apache cassandra, we must model the database after the query hence why the primary key is a composite key consisting of session_Id as the partition_key and itemin session as the clustering key for a fast retrieval

In [43]:
#query statement
query = "select artist, song, length from playlist_history where sessionid=338 and iteminsession=4"

In [44]:
#create table with condition in keyspace
query = "CREATE TABLE IF NOT EXISTS playlist_history"
query = query + "(sessionId int, itemInSession int, artist text, song text, length double, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [45]:
#insert data into table
query = "INSERT INTO playlist_history (sessionId, ItemInSession, artist, song, length)"
query = query + " VALUES (%s, %s, %s, %s, %s)"  
for index, row in df.iterrows():
    session.execute(query, ( row['sessionId'],row['itemInSession'],  row['artist'], row['song'], row['length']))

In [46]:
#run query to verify result
query = "select * from playlist_history WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.sessionid, row.iteminsession, row.artist, row.length, row.song)

338 4 Faithless 495.3073 Music Matters (Mark Knight Dub)


# Task2: Getting artist, song (sorted by item in session) and user (first name and last name) through user id  and session id.
our primary key here  is also a composite key : user id  and session id are the partition key because the data model must follow desired resultfor fast and efficient retrieval while  item in session as the clustering key which is also known as the sorting key to satisfy the query condition for the song to be sorted. 
A new column 'user' has been created in the dataframe to satisfy the query condition and work optimally in the nosql database

In [47]:
#query statement
query = "select artist, song, user from artist_by_user where userId=10 and sessionId=182"


In [50]:
#create table with condition in keyspace
query = "CREATE TABLE IF NOT EXISTS artist_by_user"
query = query + "(userId int, sessionId int, song text, itemInSession int, user text, artist text,PRIMARY KEY ((userId,sessionId),itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                 

In [51]:
#insert data into table
query = "INSERT INTO artist_by_user (userId, sessionId, itemInSession, song, user, artist)"
query = query + " VALUES (%s, %s, %s, %s, %s,%s)"
    
for index, row in df.iterrows():
    session.execute(query, (row['userId'], row['sessionId'], row['itemInSession'], row['song'], row['user'], row['artist']))

In [52]:
#run query to verify result
query = "select artist, song, user from artist_by_user where userId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print( row.artist, row.song, row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Task3: Getting every user for a particular song 
our primary key here is  a composite key : song, user and userId because using song and user alone as partition increases the danger of duplicates in case where different users with same usernames listen to the same song, the query will retrieve only one of these entries, this is olved by throwing in a column that makes the identity of the user unique, and in this case it's the user id.
A new column 'user' has been created in the dataframe to satisfy the query condition and work optimally in the nosql database

In [53]:
#query statement
query = "select song, user from users_by_song where song = 'All Hands Against His Own'"                    

In [54]:
#create table with condiiton in keyspace
query = "CREATE TABLE IF NOT EXISTS users_by_song"
query = query + "( song text,user text, userId int, artist text,PRIMARY KEY (song, user,userId ))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [55]:
#insert data into table
query = "INSERT INTO users_by_song (song, user,userId, artist)"
query = query + " VALUES (%s, %s, %s, %s)"
  
for index, row in df.iterrows():
    session.execute(query, (row['song'], row['user'], row['userId'], row['artist']))

In [56]:
#run query to verify result
query = "select song, user from users_by_song where song = 'All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print( row.user, row.song )

Jacqueline Lynch All Hands Against His Own
Sara Johnson All Hands Against His Own
Tegan Levine All Hands Against His Own


### Tables drop before closing connection

In [57]:
for t in ['playlist_history', 'artist_by_user','users_by_song']:
    query = f"DROP TABLE {t}"
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Session and cluster connection shutdown¶

In [58]:
session.shutdown()
cluster.shutdown()